In [1]:
using HTTP, DataFrames, JSON3, Dates

In [374]:
function assemble_url(base_url, data_type; countries = nothing, USstates = nothing)
    base = base_url * "vaccine/"
    if data_type == :trials
        url = base
    elseif data_type == :doses
        doses_url = base * "coverage/"
        
        if countries != nothing
            countries_url = doses_url * "countries/"
            countries == :all ? (url = countries_url) : (url = countries_url * string.(countries))
        
        elseif USstates!= nothing
            USstates_url = doses_url * "states/"
            USstates == :all ? (url = USstates_url) : (url = USstates_url * string.(USstates))
        else
            error("")
        end
        
    else
        error("")
    end
    
    return url
    
end

assemble_url (generic function with 1 method)

In [375]:
function assemble_df(get_data; all_data = true, order_by = :country, vaccine_timeline = :full)
   #order_by can be :state, :country
    
    if all_data
        
        if vaccine_timeline == :simple
            data = [Dict(order_by => j[string(order_by)], :date => date, :total_cases => cases) 
                for j in JSON3.read(get_data.body) for (date, cases) in j["timeline"]]   
        elseif vaccine_timeline == :full
            raw_df = vcat(DataFrame.(JSON3.read(get_data.body))...)
            timeline = vcat(DataFrame.(raw_df.timeline)...)
            data = hcat(select(raw_df, Not(:timeline)), timeline)    
        
        else 
            error("")
        end
        
    
    elseif data == false
                
        if vaccine_timeline == :simple
            raw_data = JSON3.read(get_data.body)
            dict = [Dict(:state => raw_data["state"], :date => date, :total_cases => cases) 
                     for (date, cases) in raw_data["timeline"]]
            data = DataFrame(dict)     
            
        elseif vaccine_timeline == :full
            raw_df = DataFrame(JSON3.read(get_data.body))
            data = hcat(select(raw_df, Not(:timeline)), DataFrame(raw_df.timeline))
        else
            error()
        end
                
    end
        
    return data
end

assemble_df (generic function with 1 method)

In [376]:
function opendisease_vaccines(data_type; countries = nothing, USstates = nothing, lastdays = nothing, vaccine_timeline = :simple)
    try    
        #vaccine_timeline can be :simple of :full
        base_url = "https://disease.sh/v3/covid-19/"
    
        if data_type == :trials
            url = assemble_url(base_url, data_type)
            get_data = HTTP.get(url)
        
            
        elseif data_type == :doses
            vaccine_timeline == :simple ? (fullData = false) : (fullData = true)
            params = Dict("lastdays" =>  lastdays, "fullData" => fullData)
            countries != nothing ? (order_by = :country) : (order_by = :state)
            (countries == :all || USstates == :all) ? (all_data = true) : (all_data = false)
            
            if countries != nothing
                if countries == :all
                    url = assemble_url(base_url, data_type, countries = countries)
                    get_data = HTTP.get(url, query = params)
                    data = assemble_df(get_data, all_data = all_data, order_by = order_by, vaccine_timeline = vaccine_timeline)
                    
                else 
                    arr_df = []
                    for country in countries
                        url = assemble_url(base_url, data_type, countries = countries[country])
                        get_data = HTTP.get(url, query = params)
                        df = assemble_df(get_data, all_data = all_data, order_by = order_by, vaccine_timeline = vaccine_timeline)
                        push!(arr_df, df)
                    end
                    data = vcat(arr_df...)
                    
                end
                                     
            elseif USstates != nothing 
                if USstates == :all
                    url = assemble_url(base_url, data_type, USstates = USstates)
                    get_data = HTTP.get(url, query = params)
                    data = assemble_df(get_data, all_data = all_data, order_by = order_by, vaccine_timeline = vaccine_timeline)  
                            
                else
                    arr_df = []
                    for state in USstates#1:length(USstates)
                        url = assemble_url(base_url, data_type, USstates = USstates[state])
                        get_data = HTTP.get(url, query = params)
                        df = assemble_df(get_data, all_data = all_data, order_by = order_by, vaccine_timeline = vaccine_timeline)
                        push!(arr_df, df)                        
                    end
                    data = vcat(arr_df...)
                end
                
            else
                error("'vaccine_timeline' must be one of :simple or :full")
            end
            
        
        else
            error("my")
        end
        
        return data
        
    catch e
        print(e)
        
    finally
    end
    
end

opendisease_vaccines (generic function with 1 method)

In [ ]:
# See opendiseas_vaccine_functions_and_examples

In [333]:
base_url = "https://disease.sh/v3/covid-19/"
url = assemble_url(base_url, :doses; USstates = :all)
lastdays = 3
vaccine_timeline = false
params = Dict("lastdays" => lastdays, "fullData" => vaccine_timeline)

Dict{String, Integer} with 2 entries:
  "lastdays" => 3
  "fullData" => false

In [334]:
get_data = HTTP.get(url, query = params);

In [337]:
raw_df = DataFrame.(JSON3.read(get_data.body))
vec_df = []

for i in 1:length(raw_df)
    timeline = copy(raw_df[i][!,:timeline][1])
    #length_df = length(raw_df[i][!,:timeline][1])
    data = collect(keys(timeline))
    total = collect(values(timeline))
    df = DataFrame(:data => data, :total => total)
    states = select(repeat!(raw_df[i], inner = length(timeline)), Not(:timeline))
    data = hcat(states, df)
    push!(vec_df, data)
end


data = vcat(vec_df...);
#dict = vcat(copy.(vec_df[1][!,:timeline])[1]...)

#timeline = copy(raw_df[1][!,:timeline][1])
#data = collect(keys(timeline))
#total = collect(values(timeline))
#DataFrame(:data => data, :total => total)

In [341]:
a = [Dict(:state => j["state"], :date => date, :total_cases => cases) 
    for j in JSON3.read(get_data.body) for (date, cases) in j["timeline"]]
DataFrame(a)

,state,date,total_cases
,String,Symbol,Int64
1,Alabama,1/1/22,5624234
2,Alabama,1/2/22,5624234
3,Alabama,1/3/22,5624234
4,Alaska,1/1/22,1024695
5,Alaska,1/2/22,1024695
6,Alaska,1/3/22,1024695
7,American Samoa,1/1/22,75819
8,American Samoa,1/2/22,75819
9,American Samoa,1/3/22,75819


In [339]:
hcat(vcat(raw_df...)

,state,timeline
,String,Object…
1,Alabama,"{\n ""1/1/22"": 5624234,\n ""1/2/22"": 5624234,\n ""1/3/22"": 5624234\n}"
2,Alabama,"{\n ""1/1/22"": 5624234,\n ""1/2/22"": 5624234,\n ""1/3/22"": 5624234\n}"
3,Alabama,"{\n ""1/1/22"": 5624234,\n ""1/2/22"": 5624234,\n ""1/3/22"": 5624234\n}"
4,Alaska,"{\n ""1/1/22"": 1024695,\n ""1/2/22"": 1024695,\n ""1/3/22"": 1024695\n}"
5,Alaska,"{\n ""1/1/22"": 1024695,\n ""1/2/22"": 1024695,\n ""1/3/22"": 1024695\n}"
6,Alaska,"{\n ""1/1/22"": 1024695,\n ""1/2/22"": 1024695,\n ""1/3/22"": 1024695\n}"
7,American Samoa,"{\n ""1/1/22"": 75819,\n ""1/2/22"": 75819,\n ""1/3/22"": 75819\n}"
8,American Samoa,"{\n ""1/1/22"": 75819,\n ""1/2/22"": 75819,\n ""1/3/22"": 75819\n}"
9,American Samoa,"{\n ""1/1/22"": 75819,\n ""1/2/22"": 75819,\n ""1/3/22"": 75819\n}"


In [236]:
base_url = "https://disease.sh/v3/covid-19/"
url = assemble_url(base_url, :doses; USstates = :all)
lastdays = 3
vaccine_timeline = true
params = Dict("lastdays" => lastdays, "fullData" => vaccine_timeline)

Dict{String, Integer} with 2 entries:
  "lastdays" => 3
  "fullData" => true

In [245]:
get_data = HTTP.get(url, query = params)
df = vcat(DataFrame.(JSON3.read(get_data.body))...)
timeline =  vcat(DataFrame.(df[:, :timeline])...)
data = hcat(select(df, Not(:timeline)), timeline);
#rename(data, :x1 => :timeline)

In [ ]:
states
    zip(states, df_vector)

In [264]:
USstates = [:illinois, :texas]
data_type = :doses
lastdays = 3
vaccine_timeline = true
params = Dict("lastdays" => lastdays, "fullData" => vaccine_timeline)
url = assemble_url(base_url, data_type; USstates = USstates[1])
get_data = HTTP.get(url, query = params)

raw_df = DataFrame(JSON3.read(get_data.body))
timeline = DataFrame(raw_df.timeline)
df = hcat(select(raw_df, Not(:timeline)), timeline)

#state = select(repeat(raw_df, inner = length(raw_df.state)), Not(:timeline))

,state,daily,dailyPerMillion,totalPerHundred,date,total
,String,Int64,Int64,Int64,String,Int64
1,Illinois,0,0,0,12/31/21,19742882
2,Illinois,0,0,0,1/1/22,19742882
3,Illinois,0,0,0,1/2/22,19742882


In [349]:
USstates = [:illinois, :texas]
data_type = :doses
lastdays = 3
vaccine_timeline = false
params = Dict("lastdays" => lastdays, "fullData" => vaccine_timeline)
url = assemble_url(base_url, data_type; USstates = USstates[1])
get_data = HTTP.get(url, query = params)

raw_df = DataFrame(JSON3.read(get_data.body))
raw_timeline = copy(raw_df.timeline[1])
date = collect(keys(raw_timeline))
total = collect(values(raw_timeline))
timeline = DataFrame(:date => date, :total => total)
state = select(repeat(raw_df, inner = length(raw_df.state)), Not(:timeline))
#df = hcat(state, timeline)
#timeline = (raw_df.timeline)
#df = hcat(select(raw_df, Not(:timeline)), timeline)

,state
,String
1,Illinois


In [361]:
USstates = [:illinois, :texas]
data_type = :doses
lastdays = 3
vaccine_timeline = true
params = Dict("lastdays" => lastdays, "fullData" => vaccine_timeline)
url = assemble_url(base_url, data_type; USstates = USstates[1])
get_data = HTTP.get(url, query = params)

raw_df = DataFrame(JSON3.read(get_data.body))
data = hcat(select(raw_df, Not(:timeline)), DataFrame(raw_df.timeline))

#timeline = DataFrame(raw_df.timeline)
#df = hcat(state, timeline)
#timeline = (raw_df.timeline)
#df = hcat(select(raw_df, Not(:timeline)), timeline)

,state,daily,dailyPerMillion,totalPerHundred,date,total
,String,Int64,Int64,Int64,String,Int64
1,Illinois,0,0,0,1/1/22,19742882
2,Illinois,0,0,0,1/2/22,19742882
3,Illinois,0,0,0,1/3/22,19742882


In [351]:
raw_df

,state,timeline
,String,Object…
1,Illinois,"{\n ""1/1/22"": 19742882,\n ""1/2/22"": 19742882,\n ""1/3/22"": 19742882\n}"


In [352]:
raw_data = JSON3.read(get_data.body)
a = [Dict(:state => raw_data["state"], :date => date, :total_cases => cases) 
     for (date, cases) in raw_data["timeline"]]
DataFrame(a)

,state,date,total_cases
,String,Symbol,Int64
1,Illinois,1/1/22,19742882
2,Illinois,1/2/22,19742882
3,Illinois,1/3/22,19742882


## Countries != nothing

In [328]:
base_url = "https://disease.sh/v3/covid-19/"
url = assemble_url(base_url, :doses; USstates = :all)
lastdays = 3
vaccine_timeline = true
params = Dict("lastdays" => lastdays, "fullData" => vaccine_timeline)
url = assemble_url(base_url, data_type; countries = :all)
get_data = HTTP.get(url, query = params)

raw_df = vcat(DataFrame.(JSON3.read(get_data.body))...)
timeline = vcat(DataFrame.(raw_df.timeline)...)
df = hcat(select(raw_df, Not(:timeline)), timeline)

,country,daily,dailyPerMillion,date,total,totalPerHundred
,String,Int64,Int64,String,Int64,Int64
1,Afghanistan,0,0,1/1/22,5228706,0
2,Afghanistan,0,0,1/2/22,5228706,0
3,Afghanistan,0,0,1/3/22,5228706,0
4,Albania,0,0,1/1/22,2316606,0
5,Albania,0,0,1/2/22,2316606,0
6,Albania,0,0,1/3/22,2316606,0
7,Algeria,0,0,1/1/22,12545356,0
8,Algeria,0,0,1/2/22,12545356,0
9,Algeria,0,0,1/3/22,12545356,0


In [344]:
base_url = "https://disease.sh/v3/covid-19/"
url = assemble_url(base_url, :doses; USstates = :all)
lastdays = 3
vaccine_timeline = false
params = Dict("lastdays" => lastdays, "fullData" => vaccine_timeline)
url = assemble_url(base_url, data_type; countries = :all)
get_data = HTTP.get(url, query = params)


raw_df = DataFrame.(JSON3.read(get_data.body))
                       
arr_df = []
for i in 1:length(raw_df)
    raw_timeline = copy(raw_df[i][!,:timeline][1])
    date = collect(keys(raw_timeline))
    total = collect(values(raw_timeline))
    timeline = DataFrame(:date => date, :total => total)
    states = select(repeat!(raw_df[i], inner = length(raw_timeline)), Not(:timeline))
    df = hcat(states, timeline)
    push!(arr_df, df)
end
data = vcat(arr_df...);


In [346]:
data = [Dict(:country => j["country"], :date => date, :total_cases => cases) 
    for j in JSON3.read(get_data.body) for (date, cases) in j["timeline"]]

651-element Vector{Dict{Symbol, Any}}:
 Dict(:country => "Afghanistan", :date => Symbol("1/1/22"), :total_cases => 5228706)
 Dict(:country => "Afghanistan", :date => Symbol("1/2/22"), :total_cases => 5228706)
 Dict(:country => "Afghanistan", :date => Symbol("1/3/22"), :total_cases => 5228706)
 Dict(:country => "Albania", :date => Symbol("1/1/22"), :total_cases => 2316606)
 Dict(:country => "Albania", :date => Symbol("1/2/22"), :total_cases => 2316606)
 Dict(:country => "Albania", :date => Symbol("1/3/22"), :total_cases => 2316606)
 Dict(:country => "Algeria", :date => Symbol("1/1/22"), :total_cases => 12545356)
 Dict(:country => "Algeria", :date => Symbol("1/2/22"), :total_cases => 12545356)
 Dict(:country => "Algeria", :date => Symbol("1/3/22"), :total_cases => 12545356)
 Dict(:country => "Andorra", :date => Symbol("1/1/22"), :total_cases => 115709)
 Dict(:country => "Andorra", :date => Symbol("1/2/22"), :total_cases => 115709)
 Dict(:country => "Andorra", :date => Symbol("1/3/22"), :

In [305]:
arr_df

Any[]

In [308]:
raw_timeline = copy(raw_df[1][!,:timeline][1])
length(raw_timeline)

3